In [14]:
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
import dask.distributed
from shapely.geometry import Point, Polygon
from scipy.spatial import cKDTree

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [17]:
gdf_osyu_100m_mesh = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_osyu_100m_mesh.shp')

In [18]:
#岩手県の線路データをダウンロード
gdf_width = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/数値地図/道路/2021/RdCL/3.shp', encoding='shift-jis') #このデータは道路中心線

In [19]:
#座標系を変更
src_proj = 6668 # 変換前の座標系を指定
dst_proj = 6678 # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_width.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_width = gdf_width.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [20]:
#gdf_widthを高速道路のみに絞る
gdf_width_highway = gdf_width[gdf_width['rdCtg'] == '高速自動車国道等']

# #gdf_osyu_width_highwayのindexを並び替える
gdf_width_highway.reset_index(drop=True, inplace=True)

In [21]:
#高速道路のデータフレームの頭にhighwayをつける

#すべてのカラムを取得
highway_columns = gdf_width_highway.columns.to_list()

#ループで各カラムを取得
for i in highway_columns[:22]:
  new_column = 'highway_' + i
  gdf_width_highway.rename(columns={f'{i}': f'{new_column}'}, inplace=True)

<ipython-input-21-7d84906274ee>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_width_highway.rename(columns={f'{i}': f'{new_column}'}, inplace=True)
<ipython-input-21-7d84906274ee>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_width_highway.rename(columns={f'{i}': f'{new_column}'}, inplace=True)
<ipython-input-21-7d84906274ee>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_width_highway.rename(columns={f'{i}': f'{new

In [22]:
#重心座標のPointを100mメッシュのデータフレームに追加
gdf_osyu_100m_mesh['center_geometry'] = [Point(x) for x in zip(gdf_osyu_100m_mesh['longitude'], gdf_osyu_100m_mesh['latitude'])]

In [23]:
# cKDTreeを使用して最近傍と距離を求める
line_coords = gdf_width_highway.geometry.apply(lambda geom: list(geom.coords[0]))
point_coords = gdf_osyu_100m_mesh.center_geometry.apply(lambda geom: list(geom.coords[0]))

line_tree = cKDTree(line_coords.tolist())
point_tree = cKDTree(point_coords.tolist())

# 最近傍の検索
distances, line_indices = line_tree.query(point_coords.tolist(), k=1)
nearest_lines = gdf_width_highway.iloc[line_indices]

# 距離を結果に追加
nearest_lines['distance_highway'] = distances

# 結果を表示
print(nearest_lines)

#lineのgeometryxはいらないので削除
nearest_lines.drop('geometry', axis=1, inplace=True)

#100mメッシュに対して同じ高速道路が最近傍だとindexが複数できてしまうので並び替え
nearest_lines.reset_index(drop=True, inplace=True)

#100mメッシュのデータフレームに最近棒の高速道路を追加
gdf_highway = gdf_osyu_100m_mesh.join(nearest_lines)

            highway_rID highway_lfSpanFr  highway_lfSpanTo  highway_tmpFlg  \
2064  50751-14406-i-168       2020-09-30               NaN               0   
2064  50751-14406-i-168       2020-09-30               NaN               0   
2064  50751-14406-i-168       2020-09-30               NaN               0   
2064  50751-14406-i-168       2020-09-30               NaN               0   
2064  50751-14406-i-168       2020-09-30               NaN               0   
...                 ...              ...               ...             ...   
2064  50751-14406-i-168       2020-09-30               NaN               0   
2064  50751-14406-i-168       2020-09-30               NaN               0   
2064  50751-14406-i-168       2020-09-30               NaN               0   
2064  50751-14406-i-168       2020-09-30               NaN               0   
2064  50751-14406-i-168       2020-09-30               NaN               0   

     highway_orgGILvl highway_ftCode highway_admCode highway_de

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
<ipython-input-23-6f26e5c9eb02>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearest_lines.drop('geometry', axis=1, inplace=True)


In [24]:
#座標系を変更
src_proj = 6678 # 変換前の座標系を指定
dst_proj = 4326  # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_highway.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_highway = gdf_highway.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [25]:
gdf_highway

,Meshcode,lu_code,lu_year,longitude,latitude,geometry,center_geometry,highway_rID,highway_lfSpanFr,highway_lfSpanTo,highway_tmpFlg,highway_orgGILvl,highway_ftCode,highway_admCode,highway_devDate,highway_type,highway_rdCtg,highway_state,highway_lvOrder,highway_name,highway_admOfcRd,highway_rnkWidth,highway_Width,highway_sectID,highway_tollSect,highway_medSect,highway_motorway,highway_repLtdLvl,highway_rtCode,distance_highway
0,5840378485,0500,20161019,140.931875,38.990417,"POLYGON ((140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035))",POINT (140.931875 38.99041666666667),50751-14406-i-168,2020-09-30,NaN,0,25000,2701,NaN,NaN,通常部,高速自動車国道等,通常部,0,NaN,NaN,5.5m-13m未満,NaN,NaN,有料,0,1,25000,NaN,12901.285576
1,5840378486,0500,20161019,140.933125,38.990417,"POLYGON ((140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035))",POINT (140.93312500000002 38.99041666666667),50751-14406-i-168,2020-09-30,NaN,0,25000,2701,NaN,NaN,通常部,高速自動車国道等,通常部,0,NaN,NaN,5.5m-13m未満,NaN,NaN,有料,0,1,25000,NaN,12901.284349
2,5840378487,0500,20161019,140.934375,38.990417,"POLYGON ((140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035))",POINT (140.934375 38.99041666666667),50751-14406-i-168,2020-09-30,NaN,0,25000,2701,NaN,NaN,通常部,高速自動車国道等,通常部,0,NaN,NaN,5.5m-13m未満,NaN,NaN,有料,0,1,25000,NaN,12901.283122
3,5840378495,0500,20161019,140.931875,38.991250,"POLYGON ((140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035))",POINT (140.931875 38.991249999999994),50751-14406-i-168,2020-09-30,NaN,0,25000,2701,NaN,NaN,通常部,高速自動車国道等,通常部,0,NaN,NaN,5.5m-13m未満,NaN,NaN,有料,0,1,25000,NaN,12901.285416
4,5840378496,0500,20161019,140.933125,38.991250,"POLYGON ((140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035, 140.83498 40.00035))",POINT (140.93312500000002 38.991249999999994),50751-14406-i-168,2020-09-30,NaN,0,25000,2701,NaN,NaN,通常部,高速自動車国道等,通常部,0,NaN,NaN,5.5m-13m未満,NaN,NaN,有料,0,1,25000,NaN,12901.284189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101381,5841728323,0500,20160611,141.291875,39.318750,"POLYGON ((140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035))",POINT (141.291875 39.31875),50751-14406-i-168,2020-09-30,NaN,0,25000,2701,NaN,NaN,通常部,高速自動車国道等,通常部,0,NaN,NaN,5.5m-13m未満,NaN,NaN,有料,0,1,25000,NaN,12900.869380
101382,5841728324,0500,20160611,141.293125,39.318750,"POLYGON ((140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035))",POINT (141.293125 39.31875),50751-14406-i-168,2020-09-30,NaN,0,25000,2701,NaN,NaN,通常部,高速自動車国道等,通常部,0,NaN,NaN,5.5m-13m未満,NaN,NaN,有料,0,1,25000,NaN,12900.868153
101383,5841730000,0500,20160611,141.375625,39.250417,"POLYGON ((140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035))",POINT (141.375625 39.25041666666666),50751-14406-i-168,2020-09-30,NaN,0,25000,2701,NaN,NaN,通常部,高速自動車国道等,通常部,0,NaN,NaN,5.5m-13m未満,NaN,NaN,有料,0,1,25000,NaN,12900.800260
101384,5841730001,0500,20160611,141.376875,39.250417,"POLYGON ((140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035, 140.83499 40.00035))",POINT (141.37687499999998 39.25041666666666),50751-14406-i-168,2020-09-30,NaN,0,25000,2701,NaN,NaN,通常部,高速自動車国道等,通常部,0,NaN,NaN,5.5m-13m未満,NaN,NaN,有料,0,1,25000,NaN,12900.799033


In [32]:
# # #geometryが二つあると出力できないのでcenter_geometryを削除
# gdf_highway.drop(columns=['lu_code', 'lu_year', 'longitude', 'latitude', 'geometry', 'center_geometry'], axis=1, inplace=True)

gdf_highway.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市 最近傍高速道路/奥州市_最近傍高速道路.csv', encoding='shift-jis')

In [ ]:
gdf_center = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市　重心/gdf_osyu_100m_mesh.shp')

In [ ]:
gdf_center.drop(columns=['lu_code', 'lu_year', 'geometry'], axis=1, inplace=True)

In [ ]:
#重心座標のPointを100mメッシュのデータフレームに追加
gdf_center['geometry'] = [Point(x) for x in zip(gdf_center['longitude'], gdf_center['latitude'])]

In [ ]:
gdf_center.to_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市　重心/gdf_center.shp')